# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）※本問題では未使用
* 判定は **ID 基準**（主キー `(x,y,z)`）、表示は仕様どおりの列名と順序

## 1) 問題

* `三つの線分 (x, y, z) が三角形を成すか判定し、Yes/No を返す。`
* 入力: `Triangle(x int, y int, z int)`（主キー `(x,y,z)`）
* 出力: `x, y, z, triangle`（`triangle` は `'Yes' | 'No'`）

## 2) 最適解（単一クエリ）

> 三角不等式は辺を並べると **最小＋中間 > 最大** の 1 本で判定できます。
> LEAST/GREATEST は 1 回ずつに抑え、中間値は合計から引いて求めます。
> ついでに **正の長さ** のみを許容し、加算は `bigint` で安全に比較します。

```sql
WITH sides AS (
  SELECT
    x, y, z,
    LEAST(x::bigint, y::bigint, z::bigint) AS a,        -- 最小
    GREATEST(x::bigint, y::bigint, z::bigint) AS c,     -- 最大
    (x::bigint + y::bigint + z::bigint)
      - LEAST(x::bigint, y::bigint, z::bigint)
      - GREATEST(x::bigint, y::bigint, z::bigint) AS b  -- 中間
  FROM Triangle
)
SELECT
  x, y, z,
  CASE
    WHEN x > 0 AND y > 0 AND z > 0
         AND (a::bigint + b::bigint) > c::bigint
      THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM sides;

Runtime 181 ms
Beats 62.83%

```

### 代替（配列で「並べ替え」を明示）

> `unnest(... ORDER BY ...)` で配列をソートしてから同じ判定。
> 可読性を優先したい場合に有効です（コストはほぼ同等）。

```sql
WITH sorted AS (
  SELECT
    x, y, z,
    ARRAY(SELECT u FROM unnest(ARRAY[x, y, z]) AS u ORDER BY u) AS s
  FROM Triangle
)
SELECT
  x, y, z,
  CASE
    WHEN x > 0 AND y > 0 AND z > 0
         AND (s[1]::bigint + s[2]::bigint) > s[3]::bigint
      THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM sorted;

Runtime 199 ms
Beats 29.67%

```

## 3) 要点解説

* **方針**: `a ≤ b ≤ c` に整列して **`a + b > c`** を満たせば三角形。
  行独立の判定なのでウィンドウ関数は不要。
* **オーバーフロー耐性**: `int + int` で境界を踏む懸念がある場合、`bigint` キャストで比較。
  さらに厳密にするなら `a > c - b` と書く手もあります（本件は `bigint` 比較で十分）。
* **データ品質**: 物理長さの前提上、**正の値のみ**を許容（`x>0 AND y>0 AND z>0`）。
  入力が保証されるならこの条件は省いても可。

## 4) 計算量（概算）

* 各行で定数回の算術・比較のみ：**O(N)**。
  `LEAST/GREATEST` も 1 回ずつに抑えているため軽量。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Triangle x y z]
  B[最小 最大 中間 を算出]
  C[正の長さと a+b>c を判定]
  D[出力 x y z triangle]
  A --> B
  B --> C
  C --> D
```

> まとめ：PostgreSQL では **「並べ替え＋1 比較」** が最短・明快・安全。`LEAST/GREATEST` で再計算を避け、`bigint` 比較で境界にも強くしておくと実務でも安心です。

三角不等式は行独立判定なので、これ以上は I/O や実行環境ノイズに支配されがちです。とはいえ、**堅牢性と微小な CPU 削減**の観点で、以下を押さえるとベターです。

---

## 改善ポイント（PostgreSQL 16.6 向け）

1. **キャストは一度だけ**
   `a+b>c` 比較で `bigint` に上げるのは妥当ですが、現状は `a/b/c` へ個別キャストが走ります。
   → 先に `x,y,z` を一度だけ `bigint` 化してから `LEAST/GREATEST` を取ると、キャスト回数を削減できます。

2. **LEAST/GREATEST は 1 回ずつ**
   すでに CTE で 1 回ずつに抑えられており◎。上の「先に bigint 化」と合わせると関数コールが最小化されます。

3. **AND の短絡評価を活かす並び**
   `x>0 AND y>0 AND z>0` を先に置くのは正解。`x<=0` で後段の計算をスキップできます。

4. **CTE はそのままでOK**
   PG 16 は標準で CTE をインライン化（`MATERIALIZED` 指定がなければ）します。1 本の SELECT に畳んでもプランはほぼ同等です。

---

## 微最適化版クエリ（キャスト一度・比較一度）

```sql
WITH t AS (
  SELECT
    x, y, z,
    x::bigint AS xb,
    y::bigint AS yb,
    z::bigint AS zb
  FROM Triangle
),
sides AS (
  SELECT
    x, y, z,
    LEAST(xb, yb, zb) AS a,
    GREATEST(xb, yb, zb) AS c,
    (xb + yb + zb) - LEAST(xb, yb, zb) - GREATEST(xb, yb, zb) AS b
  FROM t
)
SELECT
  x, y, z,
  CASE
    WHEN xb > 0 AND yb > 0 AND zb > 0
         AND (a + b) > c
      THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM sides;

column "xb" does not exist
LINE 20:     WHEN xb > 0 AND yb > 0 AND zb > 0
                  ^
HINT:  Perhaps you meant to reference the column "sides.x" or the column "sides.b".
```

* **キャストは先頭の一度だけ**、以降は `bigint` 同士の演算。
* 比較は **`a + b > c` の 1 回**（正の長さチェックを除く）。
* 演算回数がわずかに減るため、CPU ボトルネックの状況では 1 桁 ms 程度の改善が見込めます（環境依存）。

---

## 代替：加算オーバーフロー完全回避形（理屈の綺麗さ重視）

理屈上の安全性をさらに高めたいなら、**減算側**に寄せる手もあります（`bigint` 化しているので通常は不要）。

```sql
WITH t AS (
  SELECT x, y, z, x::bigint xb, y::bigint yb, z::bigint zb FROM Triangle
),
s AS (
  SELECT
    x, y, z,
    LEAST(xb, yb, zb) AS a,
    GREATEST(xb, yb, zb) AS c,
    (xb + yb + zb) - LEAST(xb, yb, zb) - GREATEST(xb, yb, zb) AS b
  FROM t
)
SELECT
  x, y, z,
  CASE
    WHEN xb > 0 AND yb > 0 AND zb > 0
         AND a > c - b
      THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM s;
```

---

## プロダクションで更に効かせるなら

* **生成列（STORED generated column）**: `a, b, c` あるいは `two_small_sum = a+b` を生成列化しておき、ビューから使うと再計算を回避できます（本チャレンジ環境では不可）。
* **三角形か否かで後続の絞り込みを多用**するなら、`triangle` を生成列＋部分インデックスで保持するのも手（例: `WHERE triangle='Yes'` 用）。

---

### 結論

現在の 181 ms は十分優秀です。上の「**先に bigint 化 → LEAST/GREATEST → a+b>c**」版にすると、**キャストと演算の回数が最小**になり、わずかながら安定して速く・堅牢になります。

ご指摘のとおり、`sides` CTE に `xb/yb/zb` を投影していないのに最終 SELECT で参照していたのが原因です。
以下のどちらかで解消できます。

---

## 修正案 A（`xb/yb/zb` を `sides` に残して使う）

```sql
WITH t AS (
  SELECT
    x, y, z,
    x::bigint AS xb,
    y::bigint AS yb,
    z::bigint AS zb
  FROM Triangle
),
sides AS (
  SELECT
    x, y, z,
    xb, yb, zb,
    LEAST(xb, yb, zb) AS a,
    GREATEST(xb, yb, zb) AS c,
    (xb + yb + zb) - LEAST(xb, yb, zb) - GREATEST(xb, yb, zb) AS b
  FROM t
)
SELECT
  x, y, z,
  CASE
    WHEN xb > 0 AND yb > 0 AND zb > 0
         AND (a + b) > c
      THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM sides;

Runtime 176 ms
Beats 79.94%

```

* `xb/yb/zb` を `sides` に投影しているので参照可能。

---

## 修正案 B（正の判定は `x/y/z`、大小比較は `bigint`）

```sql
WITH t AS (
  SELECT
    x, y, z,
    x::bigint AS xb,
    y::bigint AS yb,
    z::bigint AS zb
  FROM Triangle
),
sides AS (
  SELECT
    x, y, z,
    LEAST(xb, yb, zb) AS a,
    GREATEST(xb, yb, zb) AS c,
    (xb + yb + zb) - LEAST(xb, yb, zb) - GREATEST(xb, yb, zb) AS b
  FROM t
)
SELECT
  x, y, z,
  CASE
    WHEN x > 0 AND y > 0 AND z > 0
         AND (a + b) > c
      THEN 'Yes'
    ELSE 'No'
  END AS triangle
FROM sides;

Runtime 203 ms
Beats 26.04%
```

* `x/y/z` はそのまま正負チェックに使用（`int` のままで OK）。
* 加算比較は `a,b,c`（`bigint`）で実施し、境界も安全。

どちらでも同じ結果・性能特性です。シンプルさを優先するなら **B** をおすすめします。

